Возьмите регламент по проведению государственной экологической экспертизы  по ссылке https://docs.google.com/document/d/1WMp-uMA4_59Ybp2ynCvrFIPV1OwGHQA4zvW526BawJM и на его основе сделайте базу знаний для ответа на вопросы по этому регламенту.

Напишите промпт самостоятельно. Предусмотрите случаи, когда пользователи будут задавать вопросы, не касающиеся документа - на эти случаи дайте указания модели в промпте. Не забудьте установить настройки доступа на "все, у кого есть ссылка".

 Создайте нейро-консультанта по данной базе знаний. Проверьте работу созданного нейро-консультанта на самостоятельно сформулированных вопросах.

In [1]:
!pip install -q faiss-cpu==1.8.0 tiktoken==0.7.0 langchain-core==0.2.33 langchain==0.2.14 langchain_community==0.2.12 sentence_transformers mistralai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 391.5/391.5 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.0/260.0 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.0 MB/s eta 0:00:00


In [2]:
# запустите эту ячейку, если используете секретный ключ в колабе
import os
from mistralai import Mistral
from google.colab import userdata
api_key = userdata.get("MISTRAL_API_KEY")
os.environ["MISTRAL_API_KEY"] = api_key

# Создание клиента OpenAI с использованием API ключа из переменных среды
client = Mistral(api_key)

In [3]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter, MarkdownHeaderTextSplitter
import re
import requests
from sentence_transformers import SentenceTransformer
from langchain.docstore.document import Document

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [4]:
# функция для загрузки документа по ссылке из гугл драйв
def load_document_text(url: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text

    return text

In [5]:
# База знаний, которая будет подаваться в langChain
data_from_url= load_document_text('https://docs.google.com/document/d/1WMp-uMA4_59Ybp2ynCvrFIPV1OwGHQA4zvW526BawJM') # заполните аргумент функции

In [6]:
# выведем начало документа
data_from_url[:2000]

'\ufeffАдминистративный регламент\r\nФедеральной службы по надзору в сфере природопользования предоставления государственной услуги по организации и проведению государственной экологической экспертизы федерального уровня\r\nI. Общие положения\r\n*Предмет регулирования регламента*\r\n1. Административный регламент Федеральной службы по надзору в сфере природопользования предоставления государственной услуги по организации и проведению государственной экологической экспертизы федерального уровня (далее - Регламент) определяет сроки и последовательность административных процедур (действий) при предоставлении Федеральной службой по надзору в сфере природопользования (далее - Росприроднадзор) и ее территориальными органами государственной услуги по организации и проведению государственной экологической экспертизы федерального уровня (далее - государственная услуга), а также порядок взаимодействия между структурными подразделениями Росприроднадзора и территориальными органами Росприроднадзора

Предобрабатываем БЗ таким образом, чтобы можно было разметить ее в формате маркдаун разметки

In [7]:
def text_to_markdown(text):
    # Добавляем заголовок 1 уровня на основе римских чисел (без переноса строки)
    # и дублируем его строчкой ниже - иначе эта информация перенесется в метаданные, а порой она бывает полезной.
    def replace_header1(match):
        return f"# {match.group(2)}\n{match.group(2)}"

    text = re.sub(r'^(I{1,3}|IV|V)\. (.+)', replace_header1, text, flags=re.M)

    # Добавляем текст, выделенный жирным шрифтом (он заключен между *)
    # и дублируем его строчкой ниже
    def replace_header2(match):
        return f"## {match.group(1)}\n{match.group(1)}"

    text = re.sub(r'\*([^\*]+)\*', replace_header2, text)

    return text

In [8]:
markdown = text_to_markdown(data_from_url)
print(markdown[:15000])

﻿Административный регламент
Федеральной службы по надзору в сфере природопользования предоставления государственной услуги по организации и проведению государственной экологической экспертизы федерального уровня
# Общие положения
Общие положения
## Предмет регулирования регламента
Предмет регулирования регламента
1. Административный регламент Федеральной службы по надзору в сфере природопользования предоставления государственной услуги по организации и проведению государственной экологической экспертизы федерального уровня (далее - Регламент) определяет сроки и последовательность административных процедур (действий) при предоставлении Федеральной службой по надзору в сфере природопользования (далее - Росприроднадзор) и ее территориальными органами государственной услуги по организации и проведению государственной экологической экспертизы федерального уровня (далее - государственная услуга), а также порядок взаимодействия между структурными подразделениями Росприроднадзора и территориал

In [9]:
# Инструкция в system
system = load_document_text('https://docs.google.com/document/d/1zmpS7e-9weeG0wu9NRI8deeAatzmOoPeq_8LGtHBrME/edit?usp=sharing') # заполните параметр ссылкой на составленный вами промпт. Не забудьте открыть доступ

In [10]:
print(system[:1000])

﻿Ты консультант по регламенту по проведению государственной экологической экспертизы. Ответь на вопрос клиента, основываясь исключительно на предоставленной информации. Не добавляй никаких личных мнений или интерпретаций, отвечай максимально точно и подробно на основе содержимого. Не упоминай о наличии документа или источника информации. Клиент не должен знать о существовании документа, на основе которого ты формируешь ответ. Убедись, что все ответы ясны, логичны и соответствуют действующим нормам и правилам в области экологической экспертизы.


In [11]:
# делим БЗ на чанки при помощи MarkdownHeaderTextSplitter, так как предварительно мы ее разметили именно таким образом
headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2")
    ]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
fragments = markdown_splitter.split_text(markdown)

In [12]:
# выводим несколько чанков, чтобы убедиться, что все получилось в необходимом нам формате
fragments[2:4]

[Document(metadata={'Header 1': 'Общие положения', 'Header 2': 'Предмет регулирования регламента'}, page_content='Предмет регулирования регламента\n1. Административный регламент Федеральной службы по надзору в сфере природопользования предоставления государственной услуги по организации и проведению государственной экологической экспертизы федерального уровня (далее - Регламент) определяет сроки и последовательность административных процедур (действий) при предоставлении Федеральной службой по надзору в сфере природопользования (далее - Росприроднадзор) и ее территориальными органами государственной услуги по организации и проведению государственной экологической экспертизы федерального уровня (далее - государственная услуга), а также порядок взаимодействия между структурными подразделениями Росприроднадзора и территориальными органами Росприроднадзора, их должностными лицами и заявителями при предоставлении государственной услуги.'),
 Document(metadata={'Header 1': 'Общие положения', 

In [13]:
len(fragments)

41

In [14]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
# Функция для создания эмбеддингов из массива документов
def create_embeddings(documents):
    page_contents = [doc.page_content for doc in documents]
    return model.encode(page_contents)

# Создание эмбеддингов
embeddings = create_embeddings(fragments)

# Создадим индексную базу из разделенных фрагментов текста
import numpy as np
import faiss
# Преобразование эмбеддингов в массив numpy
embeddings = np.array(embeddings).astype('float32')

# Создание индекса FAISS
dimension = embeddings.shape[1]  # Размерность векторов
db = faiss.IndexFlatL2(dimension)  # Использование L2 метрики

# Добавление векторов в индекс
db.add(embeddings)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
def answer_index(system, topic, query_vector, fragments, search_index, verbose=1):

    # Поиск релевантных отрезков из базы знаний
    distances, indices =  search_index.search(query_vector, k=4)
    #indices_list = indices[0].tolist()
    # Вывод ближайших документов
    docs = [fragments[i] for i in indices[0]]
    if verbose: print('\n ===========================================: ')
    message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\nОтрывок документа №{i+1}\n=====================' + doc.page_content + '\n' for i, doc in enumerate(docs)]))
    if verbose: print('message_content :\n ======================================== \n', message_content)
    client = Mistral(api_key=api_key)
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f"Ответь на вопрос. Документ с информацией для ответа: {message_content}\n\nВопрос пользователя: \n{topic}"}
    ]

    if verbose: print('\n ===========================================: ')
    completion = client.chat.complete(
        model="mistral-small-latest",             # выберите модель самостоятельно
        messages=messages,
        temperature=0          # укажите значение
    )
    answer = completion.choices[0].message.content
    return answer  # возвращает ответ

Проверьте работу нейро-консультанта на сформулированных вами вопросах (достаточно 1-2)

In [17]:
# вопрос пользователя
topic= "Каковы основные требования к заявителю для подачи заявления на государственную экологическую экспертизу?"                                         # Напишите сюда вопрос нейро-консультанту
query_vector = model.encode([topic])
query_vector = np.array(query_vector).astype('float32')

ans = answer_index(system, topic, query_vector, fragments, db)
ans


 ===========================================: 
message_content :
 
Отрывок документа №1
=====================Прием и регистрация заявительных документов. Определение органа, уполномоченного на проведение государственной экологической экспертизы
55. Основанием для начала административной процедуры является поступление в центральный аппарат или территориальный орган Росприроднадзора, уполномоченный на проведение государственной экологической экспертизы, заявительных документов.
56. Заявители имеют право направить заявительные документы почтовым отправлением, либо предоставить их лично, либо в электронной форме с подписанием их усиленной квалифицированной электронной подписью.
57. При представлении заявительных документов лично Заявителем должностное лицо структурного подразделения Росприроднадзора (территориального органа Росприроднадзора), ответственного за работу с Заявителями, делает отметку о приеме заявления.
58. Отметка о приеме заявительных документов проставляется на копии заявл

'Основные требования к заявителю для подачи заявления на государственную экологическую экспертизу включают следующие пункты:\n\n1. **Подписание заявления**: Заявление должно быть подписано руководителем постоянно действующего единоличного исполнительного органа юридического лица, индивидуальным предпринимателем, физическим лицом или иным лицом, имеющим право действовать от имени заявителя. Полномочия этого лица должны быть подтверждены в соответствии с пунктом 2 Регламента.\n\n2. **Содержание заявления**: Заявление должно содержать следующие данные:\n   - Полное и сокращенное (если имеется) наименование юридического лица.\n   - Фамилия, имя, отчество (если имеется) индивидуального предпринимателя и физического лица.\n   - Адрес места нахождения и фактический адрес (если имеется) юридического лица.\n   - Адрес места жительства (регистрации) индивидуального предпринимателя, физического лица.\n   - Данные документа, удостоверяющего личность (индивидуального предпринимателя, физического ли

In [18]:
# вопрос пользователя
topic= "Что необходимо представить заявителю для проведения государственной экологической экспертизы?"                                         # Напишите сюда вопрос нейро-консультанту
query_vector = model.encode([topic])
query_vector = np.array(query_vector).astype('float32')

ans = answer_index(system, topic, query_vector, fragments, db)
ans


 ===========================================: 
message_content :
 
Отрывок документа №1
=====================Прием и регистрация заявительных документов. Определение органа, уполномоченного на проведение государственной экологической экспертизы
55. Основанием для начала административной процедуры является поступление в центральный аппарат или территориальный орган Росприроднадзора, уполномоченный на проведение государственной экологической экспертизы, заявительных документов.
56. Заявители имеют право направить заявительные документы почтовым отправлением, либо предоставить их лично, либо в электронной форме с подписанием их усиленной квалифицированной электронной подписью.
57. При представлении заявительных документов лично Заявителем должностное лицо структурного подразделения Росприроднадзора (территориального органа Росприроднадзора), ответственного за работу с Заявителями, делает отметку о приеме заявления.
58. Отметка о приеме заявительных документов проставляется на копии заявл

'Для проведения государственной экологической экспертизы необходимо представить заявителю следующие документы:\n\n1. **Заявительные документы**: Эти документы должны быть представлены лично или в электронной форме с подписанием усиленной квалифицированной электронной подписью. При личном представлении документов должностное лицо Росприроднадзора делает отметку о приеме заявления на копии заявления, указывая дату и время приема, фамилию, имя, отчество (при наличии) принявшего документы должностного лица, а также контактные и справочные телефоны.\n\n2. **Описание прилагаемых документов**: В случае отсутствия у заявителя копии заявления и описи прилагаемых документов, уполномоченное должностное лицо Росприроднадзора самостоятельно осуществляет копирование заявления и описи прилагаемых документов.\n\n3. **Уведомление об отказе в приеме заявительных документов**: В случае несоответствия заявительных документов требованиям, указанным в статье 11 Федерального закона № 63-ФЗ, заявителю направл

In [19]:
# вопрос пользователя
topic= "Что происходит, если заявитель не предоставляет запрошенные документы в установленный срок?"                                         # Напишите сюда вопрос нейро-консультанту
query_vector = model.encode([topic])
query_vector = np.array(query_vector).astype('float32')

ans = answer_index(system, topic, query_vector, fragments, db)
ans


 ===========================================: 
message_content :
 
Отрывок документа №1
=====================Прием и регистрация заявительных документов. Определение органа, уполномоченного на проведение государственной экологической экспертизы
55. Основанием для начала административной процедуры является поступление в центральный аппарат или территориальный орган Росприроднадзора, уполномоченный на проведение государственной экологической экспертизы, заявительных документов.
56. Заявители имеют право направить заявительные документы почтовым отправлением, либо предоставить их лично, либо в электронной форме с подписанием их усиленной квалифицированной электронной подписью.
57. При представлении заявительных документов лично Заявителем должностное лицо структурного подразделения Росприроднадзора (территориального органа Росприроднадзора), ответственного за работу с Заявителями, делает отметку о приеме заявления.
58. Отметка о приеме заявительных документов проставляется на копии заявл

'Если заявитель не предоставляет запрошенные документы в установленный срок, процесс предоставления государственной услуги может быть приостановлен или отклонен. В соответствии с предоставленной информацией, если заявитель не предоставляет необходимые документы в установленный срок, это может привести к следующим последствиям:\n\n1. **Приостановление процесса**: Если заявитель не предоставляет запрошенные документы в установленный срок, процесс предоставления государственной услуги может быть приостановлен до тех пор, пока не будут предоставлены все необходимые документы.\n\n2. **Отклонение заявления**: В случае, если заявитель не предоставляет запрошенные документы в установленный срок, его заявление может быть отклонено. В таком случае, заявителю может быть направлено уведомление об отказе в предоставлении государственной услуги.\n\n3. **Уведомление заявителя**: В случае непредоставления запрошенных документов в установленный срок, заявителю может быть направлено уведомление с требов